In [1]:
import numpy as np
import matplotlib as mpl
import uncertainty_toolbox as uct

# mpl.use("Cairo")  # for saving SVGs that Affinity Designer can parse
import matplotlib.pyplot as plt
import pathlib as pl
import dill

import candas as can
import gumbi as gmb
from candas.learn import ParameterSet

from tqdm.auto import tqdm

code_pth = pl.Path.cwd()  # for running in Jupyter
# code_pth = pl.Path(__file__)  # for running in terminal
fig_pth = code_pth.parent
data_pth = fig_pth / "data"
graph_pth = fig_pth / "graphics"
graph_pth.mkdir(exist_ok=True)

gen_pth = fig_pth / "generated"
gen_pth.mkdir(exist_ok=True)

plt.style.use(str(can.style.breve))

In [2]:
# plt.style.use('style.mplstyle')

%config InlineBackend.figure_format = 'retina'

# from utils import savefig

In [3]:
ps = ParameterSet.load(data_pth / "ADVI_ParameterSets_220528.pkl")


def make_pair(row):
    return "-".join(sorted([row.FPrimer, row.RPrimer]))


data = (
    ps.wide.query('Metric == "mean"')
    .astype({"BP": float})
    .assign(PrimerPair=lambda df: df.apply(make_pair, axis=1))
    .groupby(["Target", "PrimerPair", "Reporter"])
    .mean(numeric_only=True)
    .drop_duplicates()
    .reset_index()
)

selected = (
    data.groupby(["PrimerPair", "Reporter"])
    .size()
    .reset_index()
    .rename(columns={0: "Observations"})
    .sort_values("Observations", ascending=False)
    .reset_index(drop=True)
).iloc[[0, 1, 4, 5, 6, 8, 38, 39, 42]]

In [4]:
ds_full = gmb.DataSet(
    data=data,
    outputs=["F0_lg", "r", "K", "m"],
    log_vars=["BP", "K", "m", "r"],
    logit_vars=["GC"],
)
stdzr = ds_full.stdzr

In [5]:
static_data = data[
            ~((data.PrimerPair == 'FP004-RP004') & (data.Reporter == 'HEX'))
        ]

xval_data = data[
            (data.PrimerPair == 'FP004-RP004') & (data.Reporter == 'HEX')
        ].reset_index(drop=True)

In [6]:
from scipy.special import comb

In [7]:
J = 10
i = 3
I = len(xval_data)
    
def get_train_vec(x, i):
    """
    Get a training vector for the cross-validation
    """
    is_train = np.array([1]*i + [0]*(I-i)).astype(bool)
    np.random.RandomState(x*I+i).shuffle(is_train)
    return is_train

In [8]:
import pandas as pd
from numpy.linalg import LinAlgError

rows = []
# train_dss = []
# train_μσts = []
test_dss = []
test_μσts = []
models = []

x = 0

if (gen_pth / "Ind_model_qq.csv").exists():
    df = pd.read_csv(gen_pth / "Ind_model_qq.csv")
    print(df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist())
else:
    df = pd.DataFrame(
        columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
    ).astype({"N_train": int, "Iteration": int})

# for i in tqdm(range(1, I + 1, 5), desc="Training set size", leave=True):
i = 6
for j in tqdm(range(J), desc="Iteration", leave=False):

    so_far = df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist()
    if (i, j) in so_far:
        continue

    is_train = get_train_vec(j, i)
    train_code = "".join(is_train.astype(int).astype(str))
    k = 0
    while train_code in df["TrainCode"].values:
        is_train = get_train_vec(j * 1000 + k * I, i)
        train_code = "".join(is_train.astype(int).astype(str))
        k += 1

    train_data = xval_data.iloc[is_train]
    test_data = xval_data.iloc[~is_train]

    train_ds = gmb.DataSet(
        data=train_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    test_ds = gmb.DataSet(
        data=test_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    try:
        if train_ds.wide.shape[0] > 1:
            gp = gmb.GP(train_ds).fit(continuous_dims=["BP", "GC"], progressbar=False)
        else:
            gp = gmb.GP(train_ds)
            gp.specify_model(continuous_dims=["BP", "GC"])
            gp.filter_dims = {}
            gp.continuous_dims = ["BP", "GC"]
            gp.continuous_levels = gp._parse_levels(gp.continuous_dims, None)
            gp.continuous_coords = gp._parse_coordinates(
                gp.continuous_dims, gp.continuous_levels, None
            )
            gp.build_model()
            gp.find_MAP(progressbar=False)

        test_preds = gp.predict_points(
            gp.parray(**test_ds.wide[["BP", "GC"]].to_dict(orient="list"))
        ).get("r")
        train_preds = gp.predict_points(
            gp.parray(**train_ds.wide[["BP", "GC"]].to_dict(orient="list"))
        ).get("r")
    except LinAlgError:
        continue

    models.append(gp)
    # train_dss.append(train_ds)
    # train_μσts.append((train_preds.μ, train_preds.σ, train_ds.wide.r.values))
    test_dss.append(test_data)
    μ, σ, t = (test_preds.μ, test_preds.σ, test_ds.wide.r.values)

    test_μσts.append((μ, σ, t))

    (exp_proportions, obs_proportions) = uct.get_proportion_lists(
        μ, σ, t, prop_type="interval"
    )

    n = len(exp_proportions)
    df = pd.concat(
        [
            df,
            pd.DataFrame(
                data=np.array(
                    [
                        [i] * n,
                        [j] * n,
                        exp_proportions,
                        obs_proportions,
                        [train_code] * n,
                    ]
                ).T,
                columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
            ).astype({"N_train": int, "Iteration": int}),
        ]
    )

    df.to_csv(gen_pth / "Ind_model_qq.csv", index=False)

[(6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1),

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

In [10]:
import pandas as pd
from numpy.linalg import LinAlgError

rows = []
# train_dss = []
# train_μσts = []
test_dss = []
test_μσts = []
models = []

x = 0

if (gen_pth / "Avg_model_qq.csv").exists():
    df = pd.read_csv(gen_pth / "Avg_model_qq.csv")
    print(df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist())
else:
    df = pd.DataFrame(
        columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
    ).astype({"N_train": int, "Iteration": int})

# for i in tqdm(range(1, I + 1, 5), desc="Training set size", leave=True):
i = 6
for j in tqdm(range(J), desc="Iteration", leave=False):

    so_far = df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist()
    if (i, j) in so_far:
        continue

    is_train = get_train_vec(j, i)
    train_code = "".join(is_train.astype(int).astype(str))
    k = 0
    while train_code in df["TrainCode"].values:
        is_train = get_train_vec(j * 1000 + k * I, i)
        train_code = "".join(is_train.astype(int).astype(str))
        k += 1

    train_data = pd.concat([static_data, xval_data.iloc[is_train]])
    test_data = xval_data.iloc[~is_train]

    train_ds = gmb.DataSet(
        data=train_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    test_ds = gmb.DataSet(
        data=test_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    try:
        gp = gmb.GP(train_ds).fit(
            continuous_dims=["BP", "GC"], progressbar=False
        )
        test_preds = gp.predict_points(
            gp.parray(**test_ds.wide[["BP", "GC"]].to_dict(orient="list"))
        ).get("r")
        train_preds = gp.predict_points(
            gp.parray(**xval_data.iloc[is_train][["BP", "GC"]].to_dict(orient="list"))
        ).get("r")
    except LinAlgError:
        continue

    models.append(gp)
    # train_dss.append(train_ds)
    # train_μσts.append((train_preds.μ, train_preds.σ, train_ds.wide.r.values))
    test_dss.append(test_data)
    μ, σ, t = (test_preds.μ, test_preds.σ, test_ds.wide.r.values)

    test_μσts.append((μ, σ, t))

    (exp_proportions, obs_proportions) = uct.get_proportion_lists(
        μ, σ, t, prop_type="interval"
    )

    n = len(exp_proportions)
    df = pd.concat(
        [
            df,
            pd.DataFrame(
                data=np.array(
                    [
                        [i] * n,
                        [j] * n,
                        exp_proportions,
                        obs_proportions,
                        [train_code] * n,
                    ]
                ).T,
                columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
            ).astype({"N_train": int, "Iteration": int}),
        ]
    )

    df.to_csv(gen_pth / "Avg_model_qq.csv", index=False)

[(6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1),

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

In [11]:
1

1

In [12]:
3

3

In [13]:
import pandas as pd
from numpy.linalg import LinAlgError

rows = []
# train_dss = []
# train_μσts = []
test_dss = []
test_μσts = []
models = []

x = 0

if (gen_pth / "LMC_model_qq.csv").exists():
    df = pd.read_csv(gen_pth / "LMC_model_qq.csv")
    print(df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist())
else:
    df = pd.DataFrame(
        columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
    ).astype({"N_train": int, "Iteration": int})

# for i in tqdm(range(1, I + 1, 5), desc="Training set size", leave=True):
i = 6
for j in tqdm(range(J), desc="Iteration", leave=False):

    so_far = df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist()
    if (i, j) in so_far:
        continue

    is_train = get_train_vec(j, i)
    train_code = "".join(is_train.astype(int).astype(str))
    k = 0
    while train_code in df["TrainCode"].values:
        is_train = get_train_vec(j * 1000 + k * I, i)
        train_code = "".join(is_train.astype(int).astype(str))
        k += 1

    train_data = pd.concat([static_data, xval_data.iloc[is_train]])
    test_data = xval_data.iloc[~is_train]

    train_ds = gmb.DataSet(
        data=train_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    test_ds = gmb.DataSet(
        data=test_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    try:
        gp = gmb.GP(train_ds).fit(
            continuous_dims=["BP", "GC"],
            categorical_dims=["PrimerPair", "Reporter"],
            progressbar=False,
        )

        test_pa = gp.parray(**test_ds.wide[["BP", "GC"]].to_dict(orient="list"))
        test_pa = gp.append_categorical_points(
            test_pa, {"PrimerPair": "FP004-RP004", "Reporter": "HEX"}
        )
        test_preds = gp.predict_points(test_pa).get("r")

        train_pa = gp.parray(
            **xval_data.iloc[is_train][["BP", "GC"]].to_dict(orient="list")
        )
        train_pa = gp.append_categorical_points(
            train_pa, {"PrimerPair": "FP004-RP004", "Reporter": "HEX"}
        )
        train_preds = gp.predict_points(train_pa).get("r")
    except LinAlgError:
        continue
    
    models.append(gp)
    # train_dss.append(train_ds)
    # train_μσts.append((train_preds.μ, train_preds.σ, train_ds.wide.r.values))
    test_dss.append(test_data)
    μ, σ, t = (test_preds.μ, test_preds.σ, test_ds.wide.r.values)

    test_μσts.append((μ, σ, t))

    (exp_proportions, obs_proportions) = uct.get_proportion_lists(
        μ, σ, t, prop_type="interval"
    )

    n = len(exp_proportions)
    df = pd.concat(
        [
            df,
            pd.DataFrame(
                data=np.array(
                    [
                        [i] * n,
                        [j] * n,
                        exp_proportions,
                        obs_proportions,
                        [train_code] * n,
                    ]
                ).T,
                columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
            ).astype({"N_train": int, "Iteration": int}),
        ]
    )

    df.to_csv(gen_pth / "LMC_model_qq.csv", index=False)

[(6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1),

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
6

6

In [15]:
import pandas as pd
from numpy.linalg import LinAlgError

rows = []
# train_dss = []
# train_μσts = []
test_dss = []
test_μσts = []
models = []

x = 0

if (gen_pth / "LMC_model2_qq.csv").exists():
    df = pd.read_csv(gen_pth / "LMC_model2_qq.csv")
    print(df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist())
else:
    df = pd.DataFrame(
        columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
    ).astype({"N_train": int, "Iteration": int})

# for i in tqdm(range(1, I + 1, 5), desc="Training set size", leave=True):
i = 6
for j in tqdm(range(J), desc="Iteration", leave=False):

    so_far = df[["N_train", "Iteration"]].astype(int).apply(tuple, axis=1).tolist()
    if (i, j) in so_far:
        continue

    is_train = get_train_vec(j, i)
    train_code = "".join(is_train.astype(int).astype(str))
    k = 0
    while train_code in df["TrainCode"].values:
        is_train = get_train_vec(j * 1000 + k * I, i)
        train_code = "".join(is_train.astype(int).astype(str))
        k += 1

    train_data = pd.concat([static_data, xval_data.iloc[is_train]])
    test_data = xval_data.iloc[~is_train]

    train_ds = gmb.DataSet(
        data=train_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    test_ds = gmb.DataSet(
        data=test_data,
        outputs=["F0_lg", "r", "K", "m"],
        log_vars=["BP", "K", "m", "r"],
        logit_vars=["GC"],
        stdzr=stdzr,
    )

    try:
        gp = gmb.GP(train_ds).fit(
            continuous_dims=["BP", "GC"],
            categorical_dims=["PrimerPair"],
            progressbar=False,
        )

        test_pa = gp.parray(**test_ds.wide[["BP", "GC"]].to_dict(orient="list"))
        test_pa = gp.append_categorical_points(
            test_pa, {"PrimerPair": "FP004-RP004"}
        )
        test_preds = gp.predict_points(test_pa).get("r")

        train_pa = gp.parray(
            **xval_data.iloc[is_train][["BP", "GC"]].to_dict(orient="list")
        )
        train_pa = gp.append_categorical_points(
            train_pa, {"PrimerPair": "FP004-RP004"}
        )
        train_preds = gp.predict_points(train_pa).get("r")
    except LinAlgError:
        continue
    
    models.append(gp)
    # train_dss.append(train_ds)
    # train_μσts.append((train_preds.μ, train_preds.σ, train_ds.wide.r.values))
    test_dss.append(test_data)
    μ, σ, t = (test_preds.μ, test_preds.σ, test_ds.wide.r.values)

    test_μσts.append((μ, σ, t))

    (exp_proportions, obs_proportions) = uct.get_proportion_lists(
        μ, σ, t, prop_type="interval"
    )

    n = len(exp_proportions)
    df = pd.concat(
        [
            df,
            pd.DataFrame(
                data=np.array(
                    [
                        [i] * n,
                        [j] * n,
                        exp_proportions,
                        obs_proportions,
                        [train_code] * n,
                    ]
                ).T,
                columns=["N_train", "Iteration", "Train_pred_q", "Train_obsd_q", "TrainCode"],
            ).astype({"N_train": int, "Iteration": int}),
        ]
    )

    df.to_csv(gen_pth / "LMC_model2_qq.csv", index=False)

[(6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 0), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1), (6, 1),

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]